<a href="https://colab.research.google.com/github/Hirokazzz/DataScientistLab/blob/master/k_%E8%BF%91%E5%82%8D%E6%B3%95%E3%81%A7Iris%E3%83%87%E3%83%BC%E3%82%BF%E3%82%92%E5%88%86%E6%9E%90%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

参考文献：東京大学のデータサイエンティスト育成講座、「Pythonで始める機械学習」　O'REILY

### アイリスのクラス分類（k-近傍法）

アイリスの花弁の長さと幅、ガクの長さと幅をセンチメートル単位で観測してアイリスの分類を行います。

In [1]:
# 必要なライブラリの読み込み
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
import sys

### データの読み込み
sklearn.datasetsからIrisデータセットの読み込み

In [2]:
from sklearn.datasets import load_iris
iris_dataset = load_iris()

読み込んだデータはBunchクラスのオブジェクトであり、辞書と同じ形式のオブジェクトでキーと値を持っています。

iris_dataset.keys()でキーの一覧を確認できます。

In [3]:
print("key of iris_dataset: \n{}".format(iris_dataset.keys()))

key of iris_dataset: 
dict_keys(['data', 'target', 'target_names', 'DESCR', 'feature_names', 'filename'])


### データ詳細の確認

In [4]:
# classはIris-Setosa　、Iris-Versicolour、 Iris-Virginicaだとわかる
print(iris_dataset.DESCR) 

.. _iris_dataset:

Iris plants dataset
--------------------

**Data Set Characteristics:**

    :Number of Instances: 150 (50 in each of three classes)
    :Number of Attributes: 4 numeric, predictive attributes and the class
    :Attribute Information:
        - sepal length in cm
        - sepal width in cm
        - petal length in cm
        - petal width in cm
        - class:
                - Iris-Setosa
                - Iris-Versicolour
                - Iris-Virginica
                
    :Summary Statistics:

    ============== ==== ==== ======= ===== ====================
                    Min  Max   Mean    SD   Class Correlation
    ============== ==== ==== ======= ===== ====================
    sepal length:   4.3  7.9   5.84   0.83    0.7826
    sepal width:    2.0  4.4   3.05   0.43   -0.4194
    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)
    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)
    ============== ==== ==== ======= ===== ===========

In [5]:
# データセットの中身を確認
iris_dataset

{'DESCR': '.. _iris_dataset:\n\nIris plants dataset\n--------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 150 (50 in each of three classes)\n    :Number of Attributes: 4 numeric, predictive attributes and the class\n    :Attribute Information:\n        - sepal length in cm\n        - sepal width in cm\n        - petal length in cm\n        - petal width in cm\n        - class:\n                - Iris-Setosa\n                - Iris-Versicolour\n                - Iris-Virginica\n                \n    :Summary Statistics:\n\n    ============== ==== ==== ======= ===== ====================\n                    Min  Max   Mean    SD   Class Correlation\n    ============== ==== ==== ======= ===== ====================\n    sepal length:   4.3  7.9   5.84   0.83    0.7826\n    sepal width:    2.0  4.4   3.05   0.43   -0.4194\n    petal length:   1.0  6.9   3.76   1.76    0.9490  (high!)\n    petal width:    0.1  2.5   1.20   0.76    0.9565  (high!)\n    ===========

#### 求めたいアイリスの種類の確認
target_nameには予測しようとしている花の名前が入っている。

花の種類は3種類で「setosa」、 「versicolor」、「virginica」であることがわかる。

In [6]:
print("アイリスの種類: {}".format(iris_dataset['target_names']))

アイリスの種類: ['setosa' 'versicolor' 'virginica']


#### 特徴量の名前とデータの確認
どのようなデータがどのような順番で入っているか確認するには iris_dataset['feature_names']で取得

* 「sepal length (cm)」:ガクの長さ
*「sepal width (cm)」 :ガクの幅
*「petal length (cm)」:花弁の長さ
*「petal width (cm)」:花弁の幅

それぞれの個別の値を一部確認するには iris_dataset['data'][:5]で取得。[:5]の5の値を変更すると、その数だけ確認できます。  
全て確認する場合は、iris_dataset['data'] で取得できます。

In [7]:
print("特徴量の名前: {}".format(iris_dataset['feature_names']))

特徴量の名前: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


In [8]:
print("5件だけデータ表示: \n{}".format(iris_dataset['data'][:5]))

5件だけデータ表示: 
[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


何行何列の行列になっているかはshapeで確認できます。

データが150用意されて、特徴量が4種類あることがわかる。

In [9]:
print("Shape of data: {}".format(iris_dataset['data'].shape))

Shape of data: (150, 4)


#### データ分布の状態を確認

In [10]:
print("Target:\n{}".format(iris_dataset['target']))

Target:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


### 訓練データとテストデータの分離


In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(
  iris_dataset['data'],iris_dataset['target'],random_state=0
)

分離取得したデータの配列の形状を確認します。

In [12]:
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))

X_train shape: (112, 4)
y_train shape: (112,)


In [13]:
print("X_test shape: {}".format(X_test.shape))
print("y_test shape: {}".format(y_test.shape))

X_test shape: (38, 4)
y_test shape: (38,)


### k-近傍法
kを「１，１０，２０，３０，１００」でテストした結果10のときが良い結果となった。

In [14]:
from sklearn.neighbors import KNeighborsClassifier

k_val = [1, 10, 30, 50, 100]
for k in k_val:
  knn = KNeighborsClassifier(n_neighbors=k)
  # train学習
  knn.fit(X_train,y_train)
  # 精度
  print("Test set score{}: {:.2f}".format(k,knn.score(X_test,y_test)))
  print('-------------------------------------------------------------------')
  

Test set score1: 0.97
-------------------------------------------------------------------
Test set score10: 0.97
-------------------------------------------------------------------
Test set score30: 0.95
-------------------------------------------------------------------
Test set score50: 0.89
-------------------------------------------------------------------
Test set score100: 0.61
-------------------------------------------------------------------


### test予測
これ以降の予測はn_neighbors=10の場合の予測です。

In [ ]:
# test予測　ただしn_neighbors=50の場合
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
print("Test set predictions: \n {}".format(y_pred))

Test set predictions: 
 [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0 2 1 0 2 2 1 0
 2]


## 予測
野生のアイリスを見つけてガクと花弁の長さと幅を調べてその品種を予測する。

* ガクの長さ : 5センチ
* ガクの幅 : 2.9センチ
* 花弁の長さ : 1センチ
* 花弁の幅 : 0.2センチ

In [15]:
X_new = np.array([[5,2.9,1,0.2]])
print("X_new.shape:{}".format(X_new.shape))

X_new.shape:(1, 4)


In [16]:
prediction = knn.predict(X_new)
print("Prediction: {}".format(prediction))
print("Predicted target name: {}".format(iris_dataset['target_names'][prediction]))

Prediction: [0]
Predicted target name: ['setosa']
